### Load Libraries & Packages

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize
import pickle
import matplotlib.pyplot as plt
import time

### Import Data and State-Space Model
Let's import data and a state-space model with estimated A, B, C, and D matrices.

In [ ]:
with open('HW1_Data.pickle', 'rb') as f:
    Data = pickle.load(f)
To = Data['To'] # outdoor air temperature in degC
q_solar = Data['q_solar'] # absorbed solar radiation in W

with open('Exercise4_SS-Model.pickle', 'rb') as f:
    ss_model = pickle.load(f)
Ad = ss_model['Ad']
Bd = ss_model['Bd']
Cd = ss_model['Cd']
Dd = ss_model['Dd']

Bd_HVAC = Bd[:,:1]
Bd_dist = Bd[:,1:]
Dd_HVAC = Dd[:1]
Dd_dist = Dd[1:]

We use the same room model and outdoor temperature and solar radiation data.</br>
Our objective is optimize the HVAC output to satisfy the room setpoint temperature 22$^\circ$C. This example is just for educational purpose. In real buildings, we don't need an optimal controller for this task because a well-tuned PID controller will work well.

In [ ]:
setpoint = 22 # room setpoint temperature

Nd = To.shape[0] # number of timesteps
t_span = np.arange(0,Nd+1)
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].hlines(setpoint, 0, Nd, label='Setpoint', colors='C1', linestyle='--')
ax[0].plot(To, label='T_outdoor')
ax[0].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[0].set_ylabel('Temperature [$\degree$C]')
ax[0].set_xlabel('Time [hour]')
ax[0].legend()

ax[1].plot(q_solar, label='q_solar', color='C1')
ax[1].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[1].set_ylabel('Energy Rate [W]')
ax[1].set_xlabel('Time [hour]')
ax[1].legend()

plt.show()

### Define functions

In [ ]:
def compute_x(x0, u0_HVAC, u0_dist, 
              Ad, Bd_HVAC, Bd_dist):
    return np.dot(Ad,x0) + np.dot(Bd_HVAC,u0_HVAC) + np.dot(Bd_dist,u0_dist) # compute x[k+1] given x[k] and u[x]

def compute_y(x0, u0_HVAC, u0_dist, 
              Cd, Dd_HVAC, Dd_dist):
    return np.dot(Cd,x0) + np.dot(Dd_HVAC,u0_HVAC) + np.dot(Dd_dist,u0_dist) # compute y[k] given x[k] and u[x]




def MPC_simulation(x0,
                   Ad, Bd_HVAC, Bd_dist,
                   Cd, Dd_HVAC, Dd_dist,
                   u_dist,
                   setpoint,
                   pred_horizon):
    N = u_dist.shape[0]
    x = np.zeros((N+1,2))
    y = np.zeros((N,1))
    u_HVAC = np.zeros((N,1))
    x[0,0] = x0[0] # initial Ti
    x[0,1] = x0[1] # initial Tw
    
    # simulation
    for i in range(N):
        t = time.time()
        u_HVAC[i,:] = optimization(x0=x[i,:], # MPC optimization
                                   u_dist=u_dist[i:i+pred_horizon,:],
                                   setpoint=setpoint[i:i+pred_horizon,:])
        t_elapsed = time.time() - t
        x[i+1,:] = compute_x(x[i:i+1,:].T,
                             u_HVAC[i:i+1,:].T,
                             u_dist[i:i+1,:].T,
                             Ad, Bd_HVAC, Bd_dist).flatten()
        y[i,:] = compute_y(x[i:i+1,:].T,
                           u_HVAC[i:i+1,:].T,
                           u_dist[i:i+1,:].T,
                           Cd, Dd_HVAC, Dd_dist).flatten()
        if i%50==0:
            print('i == %d'%i)
    return x, y, u_HVAC

def optimization(x0, u_dist, setpoint):
    N = u_dist.shape[0] # number of timesteps
    
    init = np.ones(N) # initial value for the optimization below
    ans = minimize(objective, # using a 
                   init,
                   bounds=([[0,2]]*N),
                   args=(x0, N, u_dist, setpoint)
                   )
    return ans.x[0]*1000

def objective(u_HVAC, x0, N, u_dist, setpoint):
    u_HVAC = u_HVAC[:,None]*1000
    x = np.zeros((N+1,2))
    y = np.zeros((N,1))
    x[0,0] = x0[0] # initial Ti
    x[0,1] = x0[1] # initial Tw
    
    for i in range(N):
        x[i+1,:] = compute_x(x[i:i+1,:].T,
                             u_HVAC[i:i+1,:].T,
                             u_dist[i:i+1,:].T,
                             Ad, Bd_HVAC, Bd_dist).flatten()
        y[i,:] = compute_y(x[i:i+1,:].T,
                           u_HVAC[i:i+1,:].T,
                           u_dist[i:i+1,:].T,
                           Cd, Dd_HVAC, Dd_dist).flatten()
    cost = ############################ objective function ############################
    return cost

### Optimization

In [ ]:
x0 = [20,15]
u_dist = np.hstack((q_solar[:,None], To[:,None]))
setpoint_vector = np.ones((q_solar.shape[0],1))*setpoint
pred_horizon = 6 * 2

x, y, u_HVAC = MPC_simulation(x0,
                              Ad, Bd_HVAC, Bd_dist,
                              Cd, Dd_HVAC, Dd_dist,
                              u_dist,
                              setpoint_vector,
                              pred_horizon)

In [ ]:
Ti_simulation = x[:,0] # Indoor temperature 
Tw_simulation = x[:,1]

# plotting
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].plot(Ti_simulation, label='Ti_simulation', color='C0')
ax[0].hlines(setpoint, 0, Nd, label='Setpoint', colors='C1', linestyle='--')
ax[0].plot(Tw_simulation, label='Tw_simulation', color='C2')
ax[0].plot(To, label='To', color='C3')
ax[0].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[0].set_ylabel('Temperature [$\degree$C]')
ax[0].set_xlabel('Time [hour]')
ax[0].legend()

ax[1].plot(u_HVAC, label='q_HVAC_optimal')
ax[1].plot(q_solar, label='q_solar')
ax[1].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[1].set_ylabel('Energy Rate [W]')
ax[1].set_xlabel('Time [hour]')
ax[1].legend()

plt.show()